# Preparación de Datos con PySpark

Este notebook realiza la preparación de datos usando PySpark para procesar documentos PDF.

In [ ]:
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import udf, col
import numpy as np
from pdf2image import convert_from_path
import cv2

# Configurar Spark
print("Configurando Spark...")
spark = SparkSession.builder \
    .appName("PDFPreprocessing") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()
print("Spark configurado correctamente.")

# Configurar directorios
RAW_DIR = 'data/raw'
PROCESSED_DIR = 'data/processed'
os.makedirs(PROCESSED_DIR, exist_ok=True)
print(f"Directorios configurados: RAW_DIR={RAW_DIR}, PROCESSED_DIR={PROCESSED_DIR}")

In [1]:
def preprocess_image(image, target_size=(224, 224)):
    """Preprocesa una imagen para el modelo"""
    try:
        print("Iniciando preprocesamiento de imagen...")
        
        # Convertir a escala de grises
        if len(image.shape) == 3:
            print("Convirtiendo imagen a escala de grises...")
            gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            gray = image
            print("Imagen ya está en escala de grises.")
            
        # Redimensionar
        print(f"Redimensionando imagen a {target_size}...")
        resized = cv2.resize(gray, target_size)
        
        # Normalizar
        print("Normalizando imagen...")
        normalized = resized.astype(np.float32) / 255.0
        
        print("Preprocesamiento de imagen completado.")
        return normalized
    except Exception as e:
        print(f"Error preprocesando imagen: {str(e)}")
        return None

In [ ]:
def process_pdf(pdf_path):
    """Procesa un PDF y retorna información de sus páginas"""
    try:
        # Convertir PDF a imágenes
        pages = convert_from_path(pdf_path)
        pdf_name = os.path.basename(pdf_path).replace('.pdf', '')
        
        # Crear directorio para este PDF
        pdf_dir = os.path.join(PROCESSED_DIR, pdf_name)
        os.makedirs(pdf_dir, exist_ok=True)
        
        # Procesar cada página
        page_info = []
        for i, page in enumerate(pages):
            # Guardar imagen
            page_path = os.path.join(pdf_dir, f'page_{i}.png')
            page.save(page_path)
            
            # Convertir a array y preprocesar
            img_array = np.array(page)
            processed = preprocess_image(img_array)
            
            page_info.append({
                'pdf_name': pdf_name,
                'page_number': i,
                'path': page_path,
                'features': processed.flatten().tolist() if processed is not None else None
            })
            
        return page_info
    except Exception as e:
        print(f"Error procesando {pdf_path}: {str(e)}")
        return []

In [ ]:
# Schema para el DataFrame
schema = StructType([
    StructField("pdf_name", StringType(), False),
    StructField("page_number", IntegerType(), False),
    StructField("path", StringType(), False),
    StructField("features", ArrayType(FloatType()), True)
])

# Procesar PDFs y crear DataFrame
pdf_files = [f for f in os.listdir(RAW_DIR) if f.endswith('.pdf')]
all_pages = []

for pdf in pdf_files:
    pdf_path = os.path.join(RAW_DIR, pdf)
    print(f"Procesando {pdf}...")
    pages = process_pdf(pdf_path)
    all_pages.extend(pages)

# Crear DataFrame
df = spark.createDataFrame(all_pages, schema)

# Guardar DataFrame procesado
df.write.mode("overwrite").parquet(os.path.join(PROCESSED_DIR, "processed_pages.parquet"))

In [ ]:
# Mostrar estadísticas del procesamiento
print(f"Total de PDFs procesados: {len(pdf_files)}")
print(f"Total de páginas procesadas: {df.count()}")
print("\nDistribución de páginas por PDF:")
df.groupBy("pdf_name").count().show()

# Verificar valores nulos o problemas
print("\nVerificación de calidad:")
df.summary().show()

In [ ]:
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import udf, col
import numpy as np
from pdf2image import convert_from_path
import cv2

# Configurar Spark
print("Configurando Spark...")
spark = SparkSession.builder \
    .appName("PDFPreprocessing") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()
print("Spark configurado correctamente.")

# Configurar directorios
RAW_DIR = 'data/raw'
PROCESSED_DIR = 'data/processed'
os.makedirs(PROCESSED_DIR, exist_ok=True)
print(f"Directorios configurados: RAW_DIR={RAW_DIR}, PROCESSED_DIR={PROCESSED_DIR}")

def preprocess_image(image, target_size=(224, 224)):
    """Preprocesa una imagen para el modelo"""
    try:
        print("Iniciando preprocesamiento de imagen...")
        
        # Convertir a escala de grises
        if len(image.shape) == 3:
            print("Convirtiendo imagen a escala de grises...")
            gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            gray = image
            print("Imagen ya está en escala de grises.")
            
        # Redimensionar
        print(f"Redimensionando imagen a {target_size}...")
        resized = cv2.resize(gray, target_size)
        
        # Normalizar
        print("Normalizando imagen...")
        normalized = resized.astype(np.float32) / 255.0
        
        print("Preprocesamiento de imagen completado.")
        return normalized
    except Exception as e:
        print(f"Error preprocesando imagen: {str(e)}")
        return None

def process_pdf(pdf_path):
    """Procesa un PDF y retorna información de sus páginas"""
    try:
        print(f"Procesando {pdf_path}...")
        # Convertir PDF a imágenes
        pages = convert_from_path(pdf_path)
        pdf_name = os.path.basename(pdf_path).replace('.pdf', '')
        
        # Crear directorio para este PDF
        pdf_dir = os.path.join(PROCESSED_DIR, pdf_name)
        os.makedirs(pdf_dir, exist_ok=True)
        
        # Procesar cada página
        page_info = []
        for i, page in enumerate(pages):
            # Guardar imagen
            page_path = os.path.join(pdf_dir, f'page_{i}.png')
            page.save(page_path)
            
            # Convertir a array y preprocesar
            img_array = np.array(page)
            processed = preprocess_image(img_array)
            
            page_info.append({
                'pdf_name': pdf_name,
                'page_number': i,
                'path': page_path,
                'features': processed.flatten().tolist() if processed is not None else None
            })
            
        print(f"Procesamiento de {pdf_name} completado.")
        return page_info
    except Exception as e:
        print(f"Error procesando {pdf_path}: {str(e)}")
        return []

# Función UDF para Spark
@udf(returnType=ArrayType(StructType([
    StructField("pdf_name", StringType(), False),
    StructField("page_number", IntegerType(), False),
    StructField("path", StringType(), False),
    StructField("features", ArrayType(FloatType()), True)
])))
def process_pdf_udf(pdf_path):
    return process_pdf(pdf_path)

# Crear DataFrame de archivos PDF
print("Obteniendo lista de archivos PDF...")
pdf_files = [os.path.join(RAW_DIR, f) for f in os.listdir(RAW_DIR) if f.endswith('.pdf')]
pdf_df = spark.createDataFrame(pdf_files, StringType()).toDF("pdf_path")
print(f"Total de archivos PDF encontrados: {pdf_df.count()}")

# Procesar PDFs y crear DataFrame
print("Procesando PDFs con Spark...")
processed_df = pdf_df.withColumn("pages", process_pdf_udf(col("pdf_path")))
processed_df = processed_df.selectExpr("inline(pages)")
print("Procesamiento de PDFs completado.")

# Guardar DataFrame procesado
output_path = os.path.join(PROCESSED_DIR, "processed_pages.parquet")
print(f"Guardando DataFrame en {output_path}...")
processed_df.write.mode("overwrite").parquet(output_path)
print("DataFrame guardado correctamente.")

# Mostrar estadísticas del procesamiento
print(f"Total de PDFs procesados: {pdf_df.count()}")
print(f"Total de páginas procesadas: {processed_df.count()}")
print("\nDistribución de páginas por PDF:")
processed_df.groupBy("pdf_name").count().show()

# Verificar valores nulos o problemas
print("\nVerificación de calidad:")
processed_df.summary().show()